In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
import seaborn as sns

import glob, os, yaml, subprocess, itertools
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from statsmodels.distributions.empirical_distribution import ECDF
import sklearn.metrics
from sklearn.decomposition import PCA
import timeit
import scipy.stats as st
import statsmodels
import pickle, yaml, tracemalloc
from scipy.stats import binomtest

analysis_dir = '/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue'
input_data_dir = '/n/data1/hms/dbmi/farhat/ye12/who'
who_variants_combined = pd.read_csv("who_confidence_2021.csv")

import itertools
from stats_utils import *
import warnings
warnings.filterwarnings("ignore")

In [13]:
out_dir = "/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue/Rifampicin/BINARY/tiers=1+2/phenos=WHO/dropAF_noSyn_unpooled"

coef_df = pd.read_csv(os.path.join(out_dir, "regression_coef.csv"))
coef_permutation = pd.read_csv(os.path.join(out_dir, "coef_permutation.csv"))
coef_bootstrapping = pd.read_csv(os.path.join(out_dir, "coef_bootstrapping.csv"))
model_analysis_PC10 = pd.read_csv(os.path.join(out_dir, "model_analysis.csv"))
model_analysis_PC5 = pd.read_csv(os.path.join(out_dir, "model_analysis_PC5.csv"))

In [20]:
model_analysis_PC10.loc[model_analysis_PC10["mutation"].str.contains("PC")].query("BH_pval < 0.05")

,mutation,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,Odds_Ratio,OR_LB,OR_UB,confidence
47,PC8,0.109689,0.020399,0.179946,0.000,0.000000,0.0,1.115931,1.020609,1.197153,NaN
134,PC5,0.053050,-0.011157,0.114910,0.001,0.012925,1.0,1.054482,0.988905,1.121773,NaN
3590,PC7,-0.058263,-0.109561,-0.007470,0.000,0.000000,0.0,0.943402,0.896227,0.992558,NaN
3603,PC9,-0.079314,-0.134240,-0.028338,0.000,0.000000,0.0,0.923749,0.874380,0.972059,NaN
3612,PC6,-0.117939,-0.175875,-0.068281,0.000,0.000000,0.0,0.888751,0.838723,0.933998,NaN
3616,PC2,-0.158313,-0.230674,-0.082948,0.000,0.000000,0.0,0.853583,0.793999,0.920399,NaN
3617,PC1,-0.181849,-0.269084,-0.090579,0.000,0.000000,0.0,0.833727,0.764079,0.913403,NaN


In [15]:
model_analysis_PC5.loc[model_analysis_PC5["mutation"].str.contains("PC")]

,mutation,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,Odds_Ratio,OR_LB,OR_UB,confidence
48,PC0,0.105472,0.034076,0.177881,0.004,0.045747,1.0,1.111235,1.034664,1.194683,NaN
197,PC4,0.042216,-0.025143,0.106095,0.037,0.284506,1.0,1.043120,0.975171,1.111928,NaN
1299,PC3,0.003964,-0.065566,0.079678,0.443,0.710109,1.0,1.003972,0.936537,1.082938,NaN
3610,PC2,-0.171752,-0.255085,-0.102689,0.000,0.000000,0.0,0.842188,0.774850,0.902407,NaN
3612,PC1,-0.212691,-0.296676,-0.126288,0.000,0.000000,0.0,0.808406,0.743285,0.881361,NaN


In [16]:
model_analysis_PC5.query("mutation=='rpoC_p.Glu1092Asp'")

,mutation,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,Odds_Ratio,OR_LB,OR_UB,confidence
16,rpoC_p.Glu1092Asp,0.19584,0.128329,0.264205,0.0,0.0,0.0,1.216332,1.136927,1.302395,5) Not assoc w R


In [17]:
model_analysis_PC10.query("mutation=='rpoC_p.Glu1092Asp'")

,mutation,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,Odds_Ratio,OR_LB,OR_UB,confidence
13,rpoC_p.Glu1092Asp,0.212831,0.147201,0.282702,0.0,0.0,0.0,1.237176,1.158587,1.32671,5) Not assoc w R


In [4]:
coef_df.query("mutation=='rpoC_p.Glu1092Asp'")

,mutation,coef
3206,rpoC_p.Glu1092Asp,0.212831


# RIFAMPICIN TEST CASE

# 0. Original Analysis: 137 new significant resistance-associated variants

## But, there are 152 with OR > 1 and BH p-val < 0.01 that are in tier 2 genes (rpoB is the only tier 1)
## These 152 mutations were studied in the next two analyses

In [3]:
folder = "BINARY"
phenos_name = "WHO"
drug = "Rifampicin"
drug_WHO_abbr = "RIF"

# get the number of high confidence resistance-associated 
tiers1_stats = pd.read_csv(os.path.join(analysis_dir, drug, "BINARY/tiers=1/phenos=WHO/dropAF_noSyn_unpooled", "model_analysis_with_stats_PC5.csv"))
tiers12_stats = pd.read_csv(os.path.join(analysis_dir, drug, "BINARY/tiers=1+2/phenos=WHO/dropAF_noSyn_unpooled", "model_analysis_with_stats_PC5.csv"))

In [4]:
tiers12_stats.query("mutation=='rpoC_p.Glu1092Asp'")[['mutation', 'predicted_effect', 'position', 'confidence', 'Odds_Ratio',
       'pval', 'BH_pval', 'Bonferroni_pval', 'Num_Isolates', 'TP', 'FP', 'TN', 'FN', 'PPV', 'Sens', 'Spec', 'PPV_LB', 'PPV_UB']]

,mutation,predicted_effect,position,confidence,Odds_Ratio,pval,BH_pval,Bonferroni_pval,Num_Isolates,TP,FP,TN,FN,PPV,Sens,Spec,PPV_LB,PPV_UB
16,rpoC_p.Glu1092Asp,missense_variant,766645,5) Not assoc w R,1.216332,0.0,0.0,0.0,2697,1711,986,19935,8352,0.634409,0.170029,0.95287,0.615909,0.652616


# 1. Likelihood Ratio Test:

152 tier 2 variants were found significant in the first analysis. We then performed a likelihood ratio test:

A mutation is removed from the original model input matrix, and then a new model is fit. 

We then compare the log-likelihoods of the original model and the model with 1 less mutation. If the p-value is significant, it means that removing the mutation significantly changes the fit of the model. 

In [5]:
drug = "Rifampicin"
phenos_name = "WHO"
out_dir = os.path.join(analysis_dir, drug, f"BINARY/LRT/phenos={phenos_name}")

LRTresults_tier1 = pd.read_csv(os.path.join(out_dir, "tier=1_results.csv")).rename(columns={"Unnamed: 0": "mutation"})
LRTresults_tier2 = pd.read_csv(os.path.join(out_dir, "tier=2_results.csv")).rename(columns={"Unnamed: 0": "mutation"})

# remove the FULL model row, which is the first row
LRTresults_tier1 = add_pval_corrections(LRTresults_tier1.iloc[1:, ])
LRTresults_tier2 = add_pval_corrections(LRTresults_tier2.iloc[1:, ])

tier1_model_permute = pd.read_csv(os.path.join(analysis_dir, drug, f"BINARY/tiers=1/phenos={phenos_name}/dropAF_noSyn_unpooled/model_analysis.csv")).query("~mutation.str.contains('PC')")
tier2_model_permute = pd.read_csv(os.path.join(analysis_dir, drug, f"BINARY/tiers=1+2/phenos={phenos_name}/dropAF_noSyn_unpooled/model_analysis.csv")).query("~mutation.str.contains('PC')")

# remove the tier 1 genes for the purposes of this analysis
tier2_model_permute = tier2_model_permute.query("mutation not in @tier1_model_permute.mutation")

# check that all mutations in the permutation dataframe are in the LRT dataframe. The only difference should be the FULL model row
# print(set(tier1_model_permute.mutation).symmetric_difference(LRTresults_tier1.index))
# print(set(tier2_model_permute.mutation).symmetric_difference(LRTresults_tier2.index))
assert len(set(tier1_model_permute.mutation).symmetric_difference(LRTresults_tier1.mutation)) == 0
assert len(set(tier2_model_permute.mutation).symmetric_difference(LRTresults_tier2.mutation)) == 0

# combine results into a single dataframe for easy searching
tier1_model = tier1_model_permute.merge(LRTresults_tier1[["mutation", "pval", "BH_pval", "Bonferroni_pval"]].rename(columns={"pval": "LRT_pval", 
                                                                                                               "BH_pval": "LRT_BH_pval", 
                                                                                                               "Bonferroni_pval": "LRT_Bonferroni_pval"
                                                                                                              }), on="mutation", how="left")

tier2_model = tier2_model_permute.merge(LRTresults_tier2[["mutation", "pval", "BH_pval", "Bonferroni_pval"]].rename(columns={"pval": "LRT_pval", 
                                                                                                               "BH_pval": "LRT_BH_pval", 
                                                                                                               "Bonferroni_pval": "LRT_Bonferroni_pval"
                                                                                                              }), on="mutation", how="left")

print(len(tier1_model), len(tier2_model))

815 2845


In [48]:
# mutations picked up by the LRT, but not the coefficient association. 
# They are not significant in the tiers 1+2 model either 
thresh = 0.05

print(f"{len(tier1_model.query('Odds_Ratio > 1 & BH_pval < @thresh & LRT_BH_pval > @thresh'))} significant in Ridge, but not LRT")
print(f"{len(tier1_model.query('Odds_Ratio > 1 & BH_pval > @thresh & LRT_BH_pval < @thresh'))} significant LRT, but not in Ridge")
print(f"{len(tier1_model.query('Odds_Ratio > 1 & BH_pval < @thresh & LRT_BH_pval < @thresh'))} significant in both")

84 significant in Ridge, but not LRT
2 significant LRT, but not in Ridge
51 significant in both


In [59]:
thresh = 0.05
tier1_model.query('Odds_Ratio > 1 & (BH_pval < @thresh | LRT_BH_pval < @thresh)').shape

(137, 14)

In [56]:
thresh = 0.01

print(f"{len(tier2_model.query('Odds_Ratio > 1 & BH_pval < @thresh & LRT_BH_pval > @thresh'))} significant in Ridge, but not LRT")
print(f"{len(tier2_model.query('Odds_Ratio > 1 & BH_pval > @thresh & LRT_BH_pval < @thresh'))} significant in LRT, but not in Ridge")
print(f"{len(tier2_model.query('Odds_Ratio > 1 & BH_pval < @thresh & LRT_BH_pval < @thresh'))} significant in both")

101 significant in Ridge, but not LRT
0 significant in LRT, but not in Ridge
2 significant in both


In [74]:
tier1_model.query('OR_LB > 1 & (BH_pval < 0.05 | LRT_BH_pval < 0.05)').shape

(88, 14)

In [70]:
tier1_model.query('OR_LB > 1 & (BH_pval < 0.05 | LRT_BH_pval < 0.05)').confidence.value_counts(dropna=False)

3) Uncertain significance    42
1) Assoc w R                 24
2) Assoc w R - Interim       17
NaN                           5
Name: confidence, dtype: int64

In [77]:
tier2_model.query('OR_LB > 1 & (BH_pval < 0.01 | LRT_BH_pval < 0.01)').confidence.value_counts(dropna=False)

NaN                          60
3) Uncertain significance     7
5) Not assoc w R              1
Name: confidence, dtype: int64

In [80]:
tier2_model.query('OR_LB > 1 & (BH_pval < 0.01 | LRT_BH_pval < 0.01)')

,mutation,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,Odds_Ratio,OR_LB,OR_UB,confidence,LRT_pval,LRT_BH_pval,LRT_Bonferroni_pval
0,rpoC_p.Val483Gly,0.273303,0.190227,0.360053,0.0,0.0,0.0,1.314298,1.209524,1.433405,NaN,6.607943e-03,0.432871,1.000000
1,rpoC_p.Glu1092Asp,0.212831,0.147201,0.282702,0.0,0.0,0.0,1.237176,1.158587,1.326710,5) Not assoc w R,5.033098e-07,0.000716,0.001432
2,rpoC_p.Ile491Thr,0.203442,0.147522,0.258894,0.0,0.0,0.0,1.225614,1.158959,1.295496,NaN,3.421011e-05,0.024332,0.097328
3,rpoC_p.Ile491Val,0.196852,0.120055,0.284481,0.0,0.0,0.0,1.217564,1.127559,1.329073,3) Uncertain significance,8.082766e-04,0.176888,1.000000
4,rpoC_p.Asn698Ser,0.184709,0.116624,0.229714,0.0,0.0,0.0,1.202868,1.123697,1.258240,NaN,7.117452e-05,0.033749,0.202492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,rpoC_p.Glu750Gly,0.041322,0.024113,0.054451,0.0,0.0,0.0,1.042188,1.024406,1.055961,NaN,4.166183e-01,1.000000,1.000000
113,rpoA_p.Pro25Arg,0.041140,0.027696,0.055928,0.0,0.0,0.0,1.041998,1.028083,1.057522,NaN,4.516600e-01,1.000000,1.000000
115,nusG_c.-98C>T,0.041026,0.024888,0.063351,0.0,0.0,0.0,1.041879,1.025200,1.065401,NaN,4.776005e-01,1.000000,1.000000
117,rpoC_p.Arg770His,0.040799,0.026104,0.052773,0.0,0.0,0.0,1.041643,1.026448,1.054190,NaN,4.648498e-01,1.000000,1.000000


In [93]:
tiers12_stats.columns

Index(['mutation', 'predicted_effect', 'position', 'confidence', 'Odds_Ratio',
       'pval', 'BH_pval', 'Bonferroni_pval', 'Significant', 'Num_Isolates',
       'Total_Isolates', 'TP', 'FP', 'TN', 'FN', 'PPV', 'NPV', 'Sens', 'Spec',
       'LR+', 'LR-', 'PPV_LB', 'PPV_UB', 'NPV_LB', 'NPV_UB', 'Sens_LB',
       'Sens_UB', 'Spec_LB', 'Spec_UB', 'LR+_LB', 'LR+_UB', 'LR-_LB',
       'LR-_UB'],
      dtype='object')

In [ ]:
.merge(tiers12_stats, on="mutation", how="left").query("mutation=='rpoC_p.Glu1092Asp'")

In [112]:
tier2_model.query('OR_LB > 1 & (BH_pval < 0.01 & LRT_BH_pval < 0.01)').merge(tiers12_stats[['mutation', 'Num_Isolates',
       'Total_Isolates', 'TP', 'FP', 'TN', 'FN', 'PPV','PPV_LB', 'PPV_UB']], on="mutation", how='left')#.query("PPV_LB >= 0.25 & TP >= 5")

,mutation,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,Odds_Ratio,OR_LB,OR_UB,...,LRT_Bonferroni_pval,Num_Isolates,Total_Isolates,TP,FP,TN,FN,PPV,PPV_LB,PPV_UB
0,rpoC_p.Glu1092Asp,0.212831,0.147201,0.282702,0.0,0.0,0.0,1.237176,1.158587,1.326710,...,0.001432,2697,30984,1711,986,19935,8352,0.634409,0.615909,0.652616
1,Rv2752c_p.Asn30Ser,0.099783,0.046205,0.149246,0.0,0.0,0.0,1.104931,1.047289,1.160958,...,0.000421,12,30984,6,6,20915,10057,0.500000,0.210945,0.789055


In [114]:
type(0)

int

In [113]:
LogisticRegression(C=0, penalty='l2', max_iter=10000, multi_class='ovr', class_weight='balanced')

LogisticRegression(C=0, class_weight='balanced', max_iter=10000,
                   multi_class='ovr')

In [85]:
tiers12_stats.query('coef_LB > 0 & (BH_pval < 0.01 | LRT_BH_pval < 0.01)')

UndefinedVariableError: name 'coef_LB' is not defined

In [64]:
tier1_model.query('Odds_Ratio > 1.25').shape

(13, 14)

In [9]:
thresh = 0.01
tier2_model.query('Odds_Ratio > 1 & (BH_pval < @thresh & LRT_BH_pval < @thresh)')

,mutation,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,Odds_Ratio,OR_LB,OR_UB,confidence,LRT_pval,LRT_BH_pval,LRT_Bonferroni_pval
1,rpoC_p.Glu1092Asp,0.212831,0.147201,0.282702,0.0,0.0,0.0,1.237176,1.158587,1.326710,5) Not assoc w R,5.033098e-07,0.000716,0.001432
20,Rv2752c_p.Asn30Ser,0.099783,0.046205,0.149246,0.0,0.0,0.0,1.104931,1.047289,1.160958,3) Uncertain significance,1.480226e-07,0.000421,0.000421


In [10]:
# this was the other mutation they called attention to as not being resistance-associated
tier2_model.query("mutation=='glpK_p.Val192fs'")

,mutation,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,Odds_Ratio,OR_LB,OR_UB,confidence,LRT_pval,LRT_BH_pval,LRT_Bonferroni_pval
197,glpK_p.Val192fs,0.027247,-0.011605,0.064002,0.023,0.194479,1.0,1.027622,0.988462,1.066094,NaN,1.0,1.0,1.0


In [51]:
thresh = 0.05
tier1_sig_variants = tier1_model.query('Odds_Ratio > 1 & (BH_pval < @thresh | LRT_BH_pval < @thresh)')["mutation"].values

thresh = 0.01
tier2_sig_variants = tier2_model.query('Odds_Ratio > 1 & (BH_pval < @thresh | LRT_BH_pval < @thresh)')["mutation"].values
print(len(tier1_sig_variants), len(tier2_sig_variants))

combined_variants = np.concatenate([tier1_sig_variants, tier2_sig_variants])
print(len(np.unique(combined_variants)))
        
combined_variants_df = pd.DataFrame({"mutation": combined_variants, "Tier": ["1"]*len(tier1_sig_variants) + ["2"]*len(tier2_sig_variants)})

# USE THIS LIST FOR THE FINAL PREDICTION MODELS
#combined_variants_df.to_csv(os.path.join(analysis_dir, drug, "BINARY/prediction_models/ridge_AND_LRT_variants.csv"), index=False)
#combined_variants_df.to_csv(os.path.join(analysis_dir, drug, "BINARY/prediction_models/ridge_OR_LRT_variants.csv"), index=False)


137 103
240


# Final Prediction Model: For Rifampicin, Adding tier 2 genes doesn't significantly improve binary metrics

In [12]:
phenos_name = "WHO"
tiers_lst = ["1"]
predict_model_tier1 = pd.read_csv(os.path.join(analysis_dir, drug, f"tiers={'+'.join(tiers_lst)}_phenos={phenos_name}_CV_results.csv"))

tiers_lst = ["1", "2"]
predict_model_tiers12 = pd.read_csv(os.path.join(analysis_dir, drug, f"tiers={'+'.join(tiers_lst)}_phenos={phenos_name}_CV_results.csv"))

pd.DataFrame({"Metric": predict_model_tier1.columns,
              "pval": st.ttest_ind(predict_model_tier1, predict_model_tiers12, axis=0, equal_var=False, alternative='two-sided')[1]
             })

,Metric,pval
0,test_roc_auc,0.868314
1,test_accuracy,0.981951
2,test_balanced_accuracy,0.933191
3,test_spec,0.918947
4,test_sens,0.884981


# 2. Perform 5-fold cross-validation for all models to see how AUC, Sens, Spec, and Accuracy change

Each feature is removed from the model, and the model is refit using 5-fold cross-validation. 

Then we perform a Student's t-test for difference of means compared to the original model to see if the metrics change significantly

NO TIER 2 MUTATIONS ARE SIGNIFICANT IN THIS WAY, AND ONLY 2 TIER 1 MUTATIONS ARE FOUND. WAY TOO STRICT

In [39]:
del ref_tier1

In [40]:
del ref_tier2

In [32]:
tier1_metrics_CV = pd.read_csv(os.path.join(out_dir, f"tier=1_binaryMetrics_CV.csv"), index_col=[0])
tier2_metrics_CV = pd.read_csv(os.path.join(out_dir, f"tier=2_binaryMetrics_CV.csv"), index_col=[0])

def determine_binaryMetrics_significant(cv_df, thresh, alt="two-sided"):
    
    ref = cv_df.loc["FULL"]
    
    for mutation in cv_df.index.unique():

        # can also use alternative = 'greater' to see if removing the mutation causes a significant decrease in metrics. FOR TIER 1, THEY GIVE THE SAME RESULTS
        pvals = st.ttest_ind(ref.iloc[:, :-1], cv_df.loc[mutation].iloc[:, :-1], axis=0, equal_var=False, alternative='two-sided')[1]

        # any p-value is less than the threshold, print it
        if sum(pvals < thresh) > 0:
            print(f"{mutation}, {cv_df.loc[mutation, 'confidence'].unique()[0]} \n{dict(zip(cv_df.columns, np.round(pvals, 4)))}")        

In [33]:
determine_binaryMetrics_significant(tier1_metrics_CV, 0.05)

rpoB_p.Asp435Val, 1) Assoc w R 
{'test_roc_auc': 0.1056, 'test_sens': 0.0157, 'test_spec': 0.9792, 'test_accuracy': 0.0778, 'test_balanced_accuracy': 0.0369}
rpoB_p.Ser450Leu, 1) Assoc w R 
{'test_roc_auc': 0.002, 'test_sens': 0.0009, 'test_spec': 0.0128, 'test_accuracy': 0.0048, 'test_balanced_accuracy': 0.0028}


In [34]:
determine_binaryMetrics_significant(tier1_metrics_CV, 0.05, alt='greater')

rpoB_p.Asp435Val, 1) Assoc w R 
{'test_roc_auc': 0.1056, 'test_sens': 0.0157, 'test_spec': 0.9792, 'test_accuracy': 0.0778, 'test_balanced_accuracy': 0.0369}
rpoB_p.Ser450Leu, 1) Assoc w R 
{'test_roc_auc': 0.002, 'test_sens': 0.0009, 'test_spec': 0.0128, 'test_accuracy': 0.0048, 'test_balanced_accuracy': 0.0028}


# 3. Model with Interaction Terms

Create Tier 1 + Tier 2 interaction terms for mutations that are significant in both the Ridge regression and the LRT

Chose 2 variants because they were the only Tier 2 genes that were significantly associated in the Ridge regression and were significant in the LRT

In [17]:
thresh = 0.01
tier2_model.query('BH_pval < @thresh & LRT_BH_pval < @thresh')

,mutation,coef,coef_LB,coef_UB,pval,BH_pval,Bonferroni_pval,Odds_Ratio,OR_LB,OR_UB,confidence,LRT_pval,LRT_BH_pval,LRT_Bonferroni_pval
1,rpoC_p.Glu1092Asp,0.212831,0.147201,0.282702,0.0,0.0,0.0,1.237176,1.158587,1.326710,5) Not assoc w R,5.033098e-07,0.000716,0.001432
20,Rv2752c_p.Asn30Ser,0.099783,0.046205,0.149246,0.0,0.0,0.0,1.104931,1.047289,1.160958,3) Uncertain significance,1.480226e-07,0.000421,0.000421


In [18]:
thresh = 0.01
interact_mut = tier2_model.query('BH_pval < @thresh & LRT_BH_pval < @thresh')["mutation"].values
df_phenos = pd.read_csv(os.path.join(analysis_dir, drug, "phenos_binary.csv")).set_index("sample_id")
#interact_mut = ['rpoC_p.Glu1092Asp', 'Rv2752c_p.Asn30Ser']

# model_matrix = pd.read_pickle(os.path.join(analysis_dir, drug, "BINARY/tiers=1+2/phenos=WHO/dropAF_noSyn_unpooled/model_matrix.pkl"))
# interact_matrix = model_matrix.copy()

# # get a list of all pairs of mutations to interact
# interactions = list(itertools.product(*[interact_mut, interact_matrix.columns[interact_matrix.columns.str.contains("rpoB")]]))

# # add interactions
# for (mut1, mut2) in interactions:
#     interact_matrix[f"{mut1}+{mut2}"] = interact_matrix[mut1] * interact_matrix[mut2]
    
# # drop any columns with no signal
# interact_matrix = interact_matrix[interact_matrix.columns[~((interact_matrix == 0).all())]]

# assert len(set(model_matrix.columns) - set(interact_matrix.columns)) == 0
# print(len(interact_matrix), len(model_matrix))

# interact_matrix.to_pickle(os.path.join(analysis_dir, drug, f"BINARY/interaction/model_matrix_{phenos_name}.pkl"))

interact_matrix = pd.read_pickle(os.path.join(analysis_dir, drug, f"BINARY/interaction/model_matrix_{phenos_name}.pkl"))

interact_matrix = interact_matrix.merge(df_phenos, left_index=True, right_index=True, how="left")
del interact_matrix["phenotypic_category"]

interact_matrix.shape

(30984, 3737)

In [19]:
# so much co-occurrence of rpoC_p.Glu1092Asp with rpoB mutations. Can also check proportion that are high confidence rpoB mutations
for mut in interact_mut:
    print(mut, interact_matrix[interact_matrix.columns[interact_matrix.columns.str.contains(mut)]].shape[1])

rpoC_p.Glu1092Asp 125
Rv2752c_p.Asn30Ser 4


In [63]:
phenos_name = "WHO"
interact_res = pd.read_csv(os.path.join(analysis_dir, drug, f"BINARY/interaction/{phenos_name}_coef.csv"))
interact_permute = pd.read_csv(os.path.join(analysis_dir, drug, f"BINARY/interaction/{phenos_name}_coef_permutation.csv"))

# assess significance using the results of the permutation test
for i, row in interact_res.iterrows():
    # p-value is the proportion of permutation coefficients that are AT LEAST AS EXTREME as the test statistic
    if row["coef"] > 0:
        interact_res.loc[i, "pval"] = np.mean(interact_permute[row["mutation"]] >= row["coef"])
    else:
        interact_res.loc[i, "pval"] = np.mean(interact_permute[row["mutation"]] <= row["coef"])
        
interact_res = add_pval_corrections(interact_res)
interact_res["Odds_Ratio"] = np.exp(interact_res["coef"])

interact_res[["Mut1", "Mut2"]] = interact_res["mutation"].str.split("+", expand=True)
interact_res = interact_res.merge(who_variants_combined.query("drug=='RIF'"), left_on="Mut2", right_on="mutation", how="left")
del interact_res["mutation_y"]
del interact_res["drug"]
interact_res.rename(columns={"mutation_x": "mutation"}, inplace=True)

In [68]:
interact_res.query("mutation.str.contains('rpoC_p.Glu1092Asp') & BH_pval < 0.05").sort_values(["BH_pval", "Odds_Ratio"], 
                                                                                           ascending=[True, False]
                                                                                          )

,mutation,coef,pval,BH_pval,Bonferroni_pval,Odds_Ratio,Mut1,Mut2,confidence
3206,rpoC_p.Glu1092Asp,0.219309,0.000,0.000000,0.0,1.245216,rpoC_p.Glu1092Asp,None,NaN
3676,rpoC_p.Glu1092Asp+rpoB_p.Ile491Tyr,0.026609,0.000,0.000000,0.0,1.026966,rpoC_p.Glu1092Asp,rpoB_p.Ile491Tyr,3) Uncertain significance
3693,rpoC_p.Glu1092Asp+rpoB_p.Met434_Asn437delinsIle,0.018817,0.000,0.000000,0.0,1.018995,rpoC_p.Glu1092Asp,rpoB_p.Met434_Asn437delinsIle,2) Assoc w R - Interim
3623,rpoC_p.Glu1092Asp+rpoB_p.Asn260Asp,-0.027974,0.000,0.000000,0.0,0.972414,rpoC_p.Glu1092Asp,rpoB_p.Asn260Asp,NaN
3725,rpoC_p.Glu1092Asp+rpoB_p.Val170Phe,-0.057892,0.003,0.038700,1.0,0.943752,rpoC_p.Glu1092Asp,rpoB_p.Val170Phe,1) Assoc w R
3708,rpoC_p.Glu1092Asp+rpoB_p.Ser450Leu,0.057540,0.004,0.048584,1.0,1.059228,rpoC_p.Glu1092Asp,rpoB_p.Ser450Leu,1) Assoc w R


In [70]:
interact_res.query("mutation.str.contains('rpoB_p.Val170Phe')")

,mutation,coef,pval,BH_pval,Bonferroni_pval,Odds_Ratio,Mut1,Mut2,confidence
2936,rpoB_p.Val170Phe,0.191617,0.000,0.0000,0.0,1.211206,rpoB_p.Val170Phe,None,NaN
3725,rpoC_p.Glu1092Asp+rpoB_p.Val170Phe,-0.057892,0.003,0.0387,1.0,0.943752,rpoC_p.Glu1092Asp,rpoB_p.Val170Phe,1) Assoc w R


In [67]:
interact_res.query("mutation.str.contains('rpoB_p.Asn260Asp')")

,mutation,coef,pval,BH_pval,Bonferroni_pval,Odds_Ratio,Mut1,Mut2,confidence
2395,rpoB_p.Asn260Asp,-0.027974,0.0,0.0,0.0,0.972414,rpoB_p.Asn260Asp,None,NaN
3623,rpoC_p.Glu1092Asp+rpoB_p.Asn260Asp,-0.027974,0.0,0.0,0.0,0.972414,rpoC_p.Glu1092Asp,rpoB_p.Asn260Asp,NaN


In [217]:
interact_matrix.groupby("phenotype")['rpoC_p.Glu1092Asp+rpoB_p.Ser450Leu'].value_counts()

phenotype  rpoC_p.Glu1092Asp+rpoB_p.Ser450Leu
0          0.0                                   20907
           1.0                                      14
1          0.0                                    8682
           1.0                                    1381
Name: rpoC_p.Glu1092Asp+rpoB_p.Ser450Leu, dtype: int64

In [224]:
interact_matrix['rpoC_p.Glu1092Asp+rpoB_p.Ser450Leu'].mean()

0.045023237800154915

In [218]:
1381 / 1385

0.9971119133574007

In [223]:
interact_matrix['rpoC_p.Glu1092Asp'].mean()

0.08704492641363284

In [222]:
interact_matrix['rpoB_p.Ser450Leu'].mean()

0.21449780531887425

In [34]:
os.path.join()

'/n/data1/hms/dbmi/farhat/Sanjana/who-mutation-catalogue/Rifampicin/BINARY/tiers=1/phenos=WHO/dropAF_noSyn_unpooled'